In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_path = "./Data/Data_src1/train"
valid_path = "./Data/Data_src1/valid"
test_path = "./Data/Data_src1/test"

In [3]:
# Transformaitions on training set 'image augmentation' - prevent overfitting (high accuracy on training set but not on test set) 
# ImageDataGenerator - generate batches of real time data (e.g. 32 images) - applies augmentation - e.g. zoom range, shear range, horizontal flip, rescale i.e. feature scaling (0-255 here)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# flow from directory connects to directory
# Resizing images to make the model faster
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 30250 images belonging to 2 classes.


In [4]:
# No transformations for test images, but requires feature scaling
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_set = validation_datagen.flow_from_directory(valid_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 6300 images belonging to 2 classes.


In [5]:
# No transformations for test images, but requires feature scaling
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 6300 images belonging to 2 classes.


CNN DEFINITION

In [6]:
# Initializing
cnn = tf.keras.models.Sequential()

In [7]:
# Convolution
# filters = kernels, activation = relu, input shape = 64x64, 3 (rgb)
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=(64,64,3)))

In [8]:
# Pooling
# Pool size = 2,2 (frame size), stride = 2 (# pixels frame shifts to right)
# padding = 'valid' (ignore) OR same (add empty pixels to make the frame complete)
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [9]:
# Second convolutional layer
# No insput shape parameter required in second layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [10]:
# Flattening
# Put results in 1D vector
cnn.add(tf.keras.layers.Flatten())

In [11]:
# Full connection
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [12]:
# Output layer
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

CNN TRAINING

In [13]:
# compile - stochastic gradient optimizer - binary cross entropy loss - accuracy
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [16]:
# train and evaluate
cnn.fit(x=training_set, validation_data=validation_set, epochs=25)

Epoch 1/25
 65/946 [=>............................] - ETA: 4:41 - loss: 0.2249 - accuracy: 0.9183

In [ ]:
cnn.save('saved_model/cnn_25epochs_adam_bincross')